In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install tensorflow-text

     |████████████████████████████████| 4.9 MB 4.9 MB/s 


#Import and Load Data


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text 
import tensorflow as tf
train_data =pd.read_csv("/content/drive/MyDrive/Arabic dialect identification/Data/df_train_cleaned.csv")
dev_data = pd.read_csv("/content/drive/MyDrive/Arabic dialect identification/Data/df_dev_cleaned.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Arabic dialect identification/Data/df_test_cleaned.csv")

In [4]:
frames = [train_data, dev_data, test_data]
  
data = pd.concat(frames)

In [6]:
data

,label,text
0,MSA,بالاضافه لقيام معلمو الجيزه للذهاب جريده الساب...
1,MSA,بعدين والله حرام تجي تلقي واقف الاشاره بالعرض ...
2,DIAL_LEV,لمسه اليد مرتين واضحه جدا والحكم
3,DIAL_LEV,بخصوص الهاتريك عمرها راح تصير
4,DIAL_GLF,الله يجبر كسرهم ويرجع لدهم قبل بكري
...,...,...
9736,MSA,نسبه الفقر الاردن حسب الدراسات
9737,MSA,بالرغم كثره الاخطاء الطبيه الا اننا لم نسمع نق...
9738,MSA,حمي الله
9739,MSA,امل الله العلي القدير يشفي الشباب الاخوان يرحم...


#TF-IDF 

In [5]:
text = data['text'].astype('U').values.tolist()
target = data['label'].astype('U').values.tolist()

In [ ]:
text

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text,target,test_size=0.2, shuffle = True,  random_state=42)

In [25]:
X_train

['يمكن بس مستغرب',
 'رساله الحتلال سنبقي رغم انف',
 'يوجد مافيا خفيه غرضها استنزاف موارد البلد الغني بابناءه ومدخراته وجعله داءماً يحتاج وليس لديه اكتفاء اهم المساهمين ذلك واشتراكهم تلك المافيا هم كما يلي',
 'ماتنساش تاخد اخوك معاك بس اوعي حد يعرف مكانه علشان مطلوب الانتربول',
 'الذات',
 'المعلق الامازيجي بتكلم مصر بلدك ماليها البلاوي السوده مصر يابني لو متعرفش حمياك وبلدك لحد النهارده عوده فرنسا لاحتلالكم واقرا التاريخ ولا تكون كالجاهل يدعي الفكر مصر ليست منافقه وحبايبكوا المغرب الجاره واخدين اليهود مستشارين وكاتمي اسرارهم النهارده حبايب بس اوكدلك كله ينكشف الاعيب الساسه بلادكم الغراء ولن نتهمكم بالنفاق كما تتهموننا لاننا ببساطه مصريون والادب والاخلاق والحضاره',
 'راح يقلون لهم سجل اسمك وراح نتصل',
 'وخصني بها بنات',
 'حنانيك وعلي رسلك ابا حسامهكذا فهد تورد الابلمتاكد بانك لا تري الناس بعين طبعك',
 'اتمني التوفيق لنادي الرمثا',
 'مش عارف ليه عاملين مشكله وماظنش الانسحاب حل الانسحاب يعني ضعف اننا نقدر نواجه ونعمل الافضل ولا الخوف انها تكسب ويبقي الشكل مش حلو شايف المفروض تحدي مش',
 'بر

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(X_train) 
tfidf_transformer = TfidfTransformer()
X_tfidf_train = tfidf_transformer.fit_transform(X_train_counts)
X_test_counts = count_vect.transform(X_test) 
X_tfidf_test = tfidf_transformer.transform(X_test_counts)

#Multinomial naïve bayes

In [7]:
X_tfidf_train.shape

NameError: ignored

In [8]:
X_tfidf_test.shape

NameError: ignored

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_tfidf_train, y_train)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
predicted = clf.predict(X_tfidf_test)

precision = accuracy_score(predicted, y_test) * 100
print("Accuracy with Multinomial Naive Bayes: {0:.6f}".format(precision))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,predicted))

#Support Vector Machine Model

In [14]:
from sklearn import svm
svm_clf = svm.SVC()
svm_clf = svm_clf.fit(X_tfidf_train, y_train)
test_predictions = svm_clf.predict(X_tfidf_test)

In [15]:
import numpy as np
from sklearn.metrics import accuracy_score
precision = accuracy_score(test_predictions, y_test) * 100
print("Accuracy with SVM: {0:.6f}".format(precision))

Accuracy with SVM: 76.303461


In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test,test_predictions))

              precision    recall  f1-score   support

    DIAL_EGY       0.87      0.54      0.67      2323
    DIAL_GLF       0.77      0.51      0.62      4008
    DIAL_LEV       0.87      0.41      0.56      2156
         MSA       0.74      0.96      0.84     11364

    accuracy                           0.76     19851
   macro avg       0.81      0.61      0.67     19851
weighted avg       0.78      0.76      0.74     19851



# Random Forest Classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_tfidf_train, y_train)
y_pred = clf.predict(X_tfidf_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    DIAL_EGY       0.79      0.53      0.64      2323
    DIAL_GLF       0.68      0.53      0.60      4008
    DIAL_LEV       0.81      0.37      0.51      2156
         MSA       0.76      0.94      0.84     11364

    accuracy                           0.75     19851
   macro avg       0.76      0.60      0.65     19851
weighted avg       0.75      0.75      0.73     19851



# Ensemble Learning: Stacking Classifier

In [19]:
from sklearn.ensemble import StackingClassifier

estimators = [
           ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
            ('MNB',MultinomialNB() ),
        
              ]

clf = StackingClassifier(
    estimators=estimators, final_estimator=svm.SVC() )

In [20]:
print("Accuracy:",clf.fit(X_tfidf_train, y_train).score(X_tfidf_test, y_test) ) 

Accuracy: 0.7775930683592767


#Saving the Model

In [2]:
import pickle 
  
saved_model = pickle.dumps(clf) 
  
# Load the pickled model 
Model_from_pickle = pickle.loads(saved_model) 

NameError: ignored

In [26]:
sentence = ['ولن يُخرِسَ الطُغاهُ صوتي مادام قلبي نابضاَ']
X_testing_counts = count_vect.transform(sentence) 
X_tfidf_testing = tfidf_transformer.transform(X_testing_counts)

Model_from_pickle.predict(X_tfidf_testing)

array(['MSA'], dtype='<U8')

In [1]:
sentence = ["شو بدك مني"]
X_testing_counts = count_vect.transform(sentence) 
X_tfidf_testing = tfidf_transformer.transform(X_testing_counts)

Model_from_pickle.predict(X_tfidf_testing)

NameError: ignored